# Encode and model and predict

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
%run '../modules/mongodb.py'

In [25]:
mongodb = MongoDB()
mongoufc = mongodb.client()['ufc']
cur = mongoufc.processing.find({})
original = pd.DataFrame(cur)
original.drop(columns=['_id'], inplace=True)
original.head()

,r_odds,b_odds,r_ev,b_ev,winner,title_bout,weight_class,gender,no_of_rounds,b_current_lose_streak,...,r_height_cms,r_reach_cms,r_weight_lbs,r_age,b_age,empty_arena,b_match_weightclass_rank,r_match_weightclass_rank,b_pound_for_pound_rank,better_rank
0,-150,130,66.667,130,0,False,Light Heavyweight,MALE,5,0,...,187.96,193.04,205,37,29,1,10,5,<NA>,Red
1,170,-200,170.0,50,1,False,Welterweight,MALE,3,2,...,180.34,193.04,170,33,32,1,<NA>,<NA>,<NA>,neither
2,110,-130,110.0,76,1,False,Middleweight,MALE,3,1,...,190.5,195.58,205,34,32,1,<NA>,<NA>,<NA>,neither
3,-675,475,14.815,475,0,False,Lightweight,MALE,3,1,...,175.26,182.88,155,29,32,1,<NA>,<NA>,<NA>,neither
4,-135,115,74.074,115,1,False,Lightweight,MALE,3,0,...,175.26,177.8,155,28,33,1,<NA>,<NA>,<NA>,neither


In [26]:
lol = original.isna().sum()
lol[lambda x : x > 0]

Series([], dtype: int64)

In [41]:
original = original.replace(r'^\s*$', np.nan, regex=True)
original = original.replace('<NA>', np.nan, regex=True)
df = original.copy()
df.head()

,r_odds,b_odds,r_ev,b_ev,winner,title_bout,weight_class,gender,no_of_rounds,b_current_lose_streak,...,r_height_cms,r_reach_cms,r_weight_lbs,r_age,b_age,empty_arena,b_match_weightclass_rank,r_match_weightclass_rank,b_pound_for_pound_rank,better_rank
0,-150,130,66.667,130,0,False,Light Heavyweight,MALE,5,0,...,187.96,193.04,205,37,29,1,10,5,NaN,Red
1,170,-200,170.0,50,1,False,Welterweight,MALE,3,2,...,180.34,193.04,170,33,32,1,NaN,NaN,NaN,neither
2,110,-130,110.0,76,1,False,Middleweight,MALE,3,1,...,190.5,195.58,205,34,32,1,NaN,NaN,NaN,neither
3,-675,475,14.815,475,0,False,Lightweight,MALE,3,1,...,175.26,182.88,155,29,32,1,NaN,NaN,NaN,neither
4,-135,115,74.074,115,1,False,Lightweight,MALE,3,0,...,175.26,177.8,155,28,33,1,NaN,NaN,NaN,neither


In [34]:
lol = original.isna().sum() # this is normal
indexes = lol[lambda x : x > 0].index
indexes = list(indexes)
lol[lambda x : x > 0]

b_avg_sig_str_landed         898
b_avg_sig_str_pct            739
b_avg_sub_att                803
b_avg_td_landed              804
b_avg_td_pct                 813
r_avg_sig_str_landed         443
r_avg_sig_str_pct            348
r_avg_sub_att                348
r_avg_td_landed              348
r_avg_td_pct                 358
b_match_weightclass_rank    3891
r_match_weightclass_rank    3450
b_pound_for_pound_rank      4715
dtype: int64

In [39]:
avgs = list(filter(lambda x : 'avg' in x, indexes))
avgs

['b_avg_sig_str_landed',
 'b_avg_sig_str_pct',
 'b_avg_sub_att',
 'b_avg_td_landed',
 'b_avg_td_pct',
 'r_avg_sig_str_landed',
 'r_avg_sig_str_pct',
 'r_avg_sub_att',
 'r_avg_td_landed',
 'r_avg_td_pct']

In [44]:
for avg in avgs:
    df[avg] = df[avg].astype('float')
    
df[avgs].dtypes

b_avg_sig_str_landed    float64
b_avg_sig_str_pct       float64
b_avg_sub_att           float64
b_avg_td_landed         float64
b_avg_td_pct            float64
r_avg_sig_str_landed    float64
r_avg_sig_str_pct       float64
r_avg_sub_att           float64
r_avg_td_landed         float64
r_avg_td_pct            float64
dtype: object

In [45]:
df[avgs].isna().sum()

b_avg_sig_str_landed    898
b_avg_sig_str_pct       739
b_avg_sub_att           803
b_avg_td_landed         804
b_avg_td_pct            813
r_avg_sig_str_landed    443
r_avg_sig_str_pct       348
r_avg_sub_att           348
r_avg_td_landed         348
r_avg_td_pct            358
dtype: int64

In [46]:
for avg in avgs:
    df.loc[df[avg].isna(), avg] = df[avg].median()
df[avgs].isna().sum()

b_avg_sig_str_landed    0
b_avg_sig_str_pct       0
b_avg_sub_att           0
b_avg_td_landed         0
b_avg_td_pct            0
r_avg_sig_str_landed    0
r_avg_sig_str_pct       0
r_avg_sub_att           0
r_avg_td_landed         0
r_avg_td_pct            0
dtype: int64